## 챌린지
---
(EN)
- Implement a complete RAG pipeline with a Stuff Documents chain.
- You must implement the chain manually.
- Give a ConversationBufferMemory to the chain.
- Use this document to perform RAG: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- Ask the following questions to the chain:
    - Is Aaronson guilty?
    - What message did he write in the table?
    - Who is Julia?

(KR)
- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
- 체인을 수동으로 구현해야 합니다.
- 체인에 ConversationBufferMemory를 부여합니다.
- 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- 체인에 다음 질문을 합니다:
    - Aaronson 은 유죄인가요?
    - 그가 테이블에 어떤 메시지를 썼나요?
    - Julia 는 누구인가요?

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_unstructured import UnstructuredLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True
)

# 캐시 저장 경로
cache_dir = LocalFileStore("./.cache/")

# Tiktoken: OpenAI에서 만든 tokenization library.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=950,
    chunk_overlap=100,
)

loader = UnstructuredLoader('./files/document.txt')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# 중복 요청 시 캐시된 결과를 반환
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# FAISS 라이브러리로 캐시에서 임베딩 벡터 검색
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# docs를 불러오는 역할
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make it up:\n\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def prepare_inputs(inputs):
    query = inputs["question"]
    retrieved_docs = retriever.get_relevant_documents(query)
    context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    return {
        "context": context_text,
        "history": memory.load_memory_variables({})["history"],
        "question": query
    }

# CASE: chain_type = 'stuff' (default)
chain = RunnableLambda(prepare_inputs) | prompt | llm

def invoke_chain(question):
    response = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": response.content})
    print(f"{response.content}\n\n")

In [18]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


He accepted everything. The past was alterable. The past never had been altered. Oceania was at war with Eastasia. Oceania had always been at war with Eastasia. Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with. He had never seen the photograph that disproved their guilt. It had never existed, he had invented it. He remembered remembering contrary things, but those were false memories, products of self-deception. How easy it all was! Only surrender, and everything else followed. It was like swimming against a current that swept you backwards however hard you struggled, and then suddenly deciding to turn round and go with the current instead of opposing it. Nothing had changed except your own attitude: the predestined thing happened in any case. He hardly knew why he had ever rebelled. Everything was easy, except----!




INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


DOWN WITH BIG BROTHER




INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Julia is a character in the novel who is involved in a forbidden romantic relationship with the protagonist.




## 정답코드

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    print(result.content)
    memory.save_context({"input": question}, {"output": result.content})

In [5]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
He wrote "FREEDOM IS SLAVERY" and then "TWO AND TWO MAKE FIVE" on the table.
Julia is a character in the document who was involved with Winston in a forbidden relationship.


In [6]:
load_memory({})


[HumanMessage(content='Is Aaronson guilty?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What message did he write in the table?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='He wrote "FREEDOM IS SLAVERY" and then "TWO AND TWO MAKE FIVE" on the table.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Who is Julia?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Julia is a character in the document who was involved with Winston in a forbidden relationship.', additional_kwargs={}, response_metadata={})]